# ============================================================
# PART 1 — Customer Satisfaction Prediction — Full ML Pipeline
# ============================================================
# 
### By Ramjeet Dixit

This notebook implements a complete end-to-end machine learning workflow to analyze and predict airline customer satisfaction.

The goals are to:
- Understand key drivers of customer satisfaction  
- Explore demographic, operational, and service-related patterns  
- Build and compare several machine learning models  
- Identify the best-performing model for predicting satisfaction  

The workflow includes:
- Data loading and cleaning  
- Descriptive statistics and visual exploration  
- Statistical hypothesis testing (t-test, ANOVA, correlations)  
- Feature engineering and preprocessing  
- Model training, evaluation, and comparison  
- Optional hyperparameter tuning with GridSearchCV  
- Feature importance and learning curve diagnostics  

All outputs (CSV summaries, model performance tables, feature importance exports, tuning results) will be saved into an `outputs` folder on the Desktop for easy access and reporting.

In [ ]:
# ============================================================
# PART 2 — IMPORTS, GLOBAL SETTINGS, OUTPUT DIRECTORY
# ============================================================

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path

from scipy.stats import ttest_ind, f_oneway, pearsonr

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import KNNImputer

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix
)

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier

# Optional XGBoost
try:
    from xgboost import XGBClassifier
    xgb_available = True
except ImportError:
    xgb_available = False

from sklearn.model_selection import GridSearchCV, cross_val_score, learning_curve

# ------------------------------------------------------------
# GLOBAL SETTINGS
# ------------------------------------------------------------
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (8, 5)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", None)

# ------------------------------------------------------------
# FILE PATHS
# ------------------------------------------------------------

# Your dataset path
CSV_FILE_PATH = "/Users/ramjeetdixit/Desktop/Customer Satisfaction.csv"

# Output directory — always writable on macOS Desktop
OUTPUT_DIR = Path("/Users/ramjeetdixit/Desktop/outputs")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# ------------------------------------------------------------
# HELPER FUNCTION
# ------------------------------------------------------------
def section(title: str):
    print("\n" + "=" * 60)
    print(title.upper())
    print("=" * 60 + "\n")

In [ ]:
# ============================================================
# PART 3 — DATA LOADING & CLEANING (FINAL VERSION)
# ============================================================

def load_data(csv_path: str) -> pd.DataFrame:
    section("Loading Data")

    csv_path = Path(csv_path)
    if not csv_path.exists():
        raise FileNotFoundError(f"CSV file not found at {csv_path}")

    df = pd.read_csv(csv_path)
    print(f"Data loaded successfully with shape: {df.shape}")
    return df


# ------------------------------------------------------------
# BASIC CLEANING UTILITIES
# ------------------------------------------------------------

def convert_blank_to_nan(df: pd.DataFrame) -> pd.DataFrame:
    section("Converting blanks to NaN")
    return df.replace(r"^\s*$", np.nan, regex=True)


def clean_categorical(df: pd.DataFrame) -> pd.DataFrame:
    section("Cleaning categorical columns")

    cat_cols = df.select_dtypes(include=["object"]).columns

    for col in cat_cols:
        df[col] = df[col].astype(str).str.strip().str.title()

    return df


def convert_to_numeric(df: pd.DataFrame, numeric_cols: list) -> pd.DataFrame:
    section("Converting selected columns to numeric")
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    return df


def fill_missing_with_median(df: pd.DataFrame, numeric_cols: list) -> pd.DataFrame:
    section("Median imputation for numeric columns")
    for col in numeric_cols:
        df[col] = df[col].fillna(df[col].median())
    return df


# ------------------------------------------------------------
# OUTLIER HANDLING — ASSIGNMENT RULE IMPLEMENTED
# ------------------------------------------------------------

def remove_outliers_if_less_than_5_percent(df: pd.DataFrame, numeric_cols: list) -> pd.DataFrame:
    section("Outlier Detection & Conditional Removal (<5% Rule)")

    df = df.copy()
    rows_before = df.shape[0]

    for col in numeric_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1

        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR

        outliers = df[(df[col] < lower) | (df[col] > upper)]
        outlier_pct = (len(outliers) / len(df)) * 100

        print(f"\nColumn: {col}")
        print(f"Outliers detected: {len(outliers)} ({outlier_pct:.2f}%)")

        # Assignment rule:
        if outlier_pct < 5:
            print("→ Removing outliers (below 5% threshold)")
            df = df[(df[col] >= lower) & (df[col] <= upper)]
        else:
            print("→ Outliers NOT removed (above 5% threshold)")

    rows_after = df.shape[0]
    print(f"\nRows before: {rows_before}, Rows after: {rows_after}")
    print(f"Total rows removed: {rows_before - rows_after}")

    return df


# ------------------------------------------------------------
# MAIN CLEANING PIPELINE
# ------------------------------------------------------------

def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    section("Cleaning Data — Full Pipeline")

    df = df.copy()

    print("Initial shape:", df.shape)

    # Drop fully empty rows
    df.dropna(how="all", inplace=True)

    # Remove duplicates
    before = df.shape[0]
    df.drop_duplicates(inplace=True)
    after = df.shape[0]
    print(f"Removed {before - after} duplicate rows")

    # Convert blanks to NaN
    df = convert_blank_to_nan(df)

    # Clean categorical columns
    df = clean_categorical(df)

    # ------------------------------------------------------------
    # FULL NUMERIC COLUMN LIST (UPDATED)
    # ------------------------------------------------------------
    numeric_cols = [
        "Id",
        "Age",
        "Distance Travelled",
        "Departure/Arrival Rating",
        "Booking Ease",
        "Boarding Point",
        "Food",
        "Seat Comfort",
        "Entertainment",
        "Leg Room",
        "Luggage Handling",
        "Cleanliness",
        "Departure Delay (min)",
        "Arrival Delay (min)"
    ]

    # Keep only columns that exist
    numeric_cols = [c for c in numeric_cols if c in df.columns]

    # Convert to numeric
    df = convert_to_numeric(df, numeric_cols)

    # Median imputation
    df = fill_missing_with_median(df, numeric_cols)

    # Outlier removal (assignment rule)
    df = remove_outliers_if_less_than_5_percent(df, numeric_cols)

    print("\nFinal shape after cleaning:", df.shape)
    print("Cleaning complete.")

    return df

## 📥 Loading and Cleaning the Dataset

In this section, we load the airline customer satisfaction dataset from the Desktop and apply the full cleaning pipeline.

The cleaning process includes:
- Removing fully empty rows  
- Converting blank strings to `NaN`  
- Converting selected columns to numeric  
- Median imputation for missing numeric values  
- Outlier clipping (1st–99th percentile)  

After cleaning, we preview the first few rows to confirm that the dataset is ready for analysis and modeling.

In [ ]:
# ============================================================
# PART 4 — LOAD & CLEAN DATA (FINAL)
# ============================================================

# Load the dataset once
df = load_data(CSV_FILE_PATH)

# Apply full cleaning pipeline once
df = clean_data(df)

# Freeze a clean copy for descriptive statistics (Part 6)
df_clean = df.copy()

# Preview cleaned data
df.head()


In [ ]:
section("VERIFYING df_clean COLUMNS")
print(df_clean.columns.tolist())

In [ ]:
# ============================================================
# PART 5 — FEATURE ENGINEERING (FINAL)
# ============================================================

section("PART 5 — FEATURE ENGINEERING")

df_fe = df_clean.copy()   # Start from the cleaned dataset


# ------------------------------------------------------------
# 1. CREATE NEW FEATURES
# ------------------------------------------------------------
section("Creating New Features")

# Total Delay = Departure Delay + Arrival Delay
df_fe["Total_Delay"] = (
    df_fe["Departure Delay (min)"] +
    df_fe["Arrival Delay (min)"]
)

# Delay Category (optional)
df_fe["Delay_Category"] = pd.cut(
    df_fe["Total_Delay"],
    bins=[-1, 0, 30, 120, 10000],
    labels=["No Delay", "Short Delay", "Medium Delay", "Long Delay"]
)

print("New features created: Total_Delay, Delay_Category")


# ------------------------------------------------------------
# 2. ENCODE CATEGORICAL VARIABLES
# ------------------------------------------------------------
section("Encoding Categorical Variables")

categorical_cols = [
    "Gender",
    "Travel Category",
    "Travel Class",
    "Delay_Category",
    "Satisfaction"
]

# Label Encoding for ordinal-like categories
le = LabelEncoder()

for col in categorical_cols:
    if col in df_fe.columns:
        df_fe[col] = le.fit_transform(df_fe[col].astype(str))

print("Categorical variables encoded.")


# ------------------------------------------------------------
# 3. DROP IRRELEVANT COLUMNS
# ------------------------------------------------------------
section("Dropping Irrelevant Columns")

cols_to_drop = ["Id"]  # ID has no predictive value

df_fe.drop(columns=[c for c in cols_to_drop if c in df_fe.columns], inplace=True)

print("Dropped columns:", cols_to_drop)


# ------------------------------------------------------------
# 4. DEFINE FEATURES (X) AND TARGET (y)
# ------------------------------------------------------------
section("Preparing X and y")

# Target variable
y = df_fe["Satisfaction"]

# Features = everything except target
X = df_fe.drop(columns=["Satisfaction"])

print("X shape:", X.shape)
print("y shape:", y.shape)


# ------------------------------------------------------------
# 5. SCALE NUMERICAL FEATURES
# ------------------------------------------------------------
section("Scaling Numerical Features")

numeric_cols = X.select_dtypes(include=[np.number]).columns

scaler = StandardScaler()
X_scaled = X.copy()
X_scaled[numeric_cols] = scaler.fit_transform(X[numeric_cols])

print("Numerical features scaled.")


# ------------------------------------------------------------
# FINAL OUTPUT FOR MODEL TRAINING
# ------------------------------------------------------------
section("Feature Engineering Complete")

print("Feature engineering completed successfully.")
print("Dataset ready for model training in Part 7.")

# Freeze final datasets for model training
X_final = X_scaled.copy()
y_final = y.copy()

In [ ]:
# ============================================================
# PART 6 — DESCRIPTIVE STATISTICS & VISUAL ANALYSIS (OPTIMIZED)
# ============================================================

section("PART 6 — DESCRIPTIVE STATISTICS & VISUAL ANALYSIS (OPTIMIZED)")


# ------------------------------------------------------------
# 1. BASIC DESCRIPTIVE STATISTICS
# ------------------------------------------------------------

def descriptive_statistics(df: pd.DataFrame):
    section("Descriptive Statistics")

    print("Basic Info:")
    print(df.info())

    print("\nSummary Statistics:")
    summary = df.describe(include="all")
    print(summary)

    summary.to_csv(OUTPUT_DIR / "summary_statistics.csv")


# ------------------------------------------------------------
# 2. GENDER DISTRIBUTION
# ------------------------------------------------------------

def analyze_gender_distribution(df: pd.DataFrame):
    section("Gender Distribution")

    gender_counts = df["Gender"].value_counts()
    print(gender_counts)

    plt.figure(figsize=(6, 4))
    sns.countplot(data=df, x="Gender", palette="Set2")
    plt.title("Gender Distribution")
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(6, 6))
    gender_counts.plot.pie(autopct="%1.1f%%", colors=sns.color_palette("Set2"))
    plt.title("Gender Distribution (Pie Chart)")
    plt.ylabel("")
    plt.tight_layout()
    plt.show()


# ------------------------------------------------------------
# 3. AGE DISTRIBUTION
# ------------------------------------------------------------

def analyze_age_distribution(df: pd.DataFrame):
    section("Age Distribution")

    plt.figure(figsize=(8, 5))
    sns.histplot(df["Age"], bins=30, color="skyblue")
    plt.title("Age Distribution")
    plt.tight_layout()
    plt.show()

    df["Age_Group"] = pd.cut(
        df["Age"],
        bins=[0, 18, 30, 45, 60, 100],
        labels=["0-18", "19-30", "31-45", "46-60", "60+"]
    )

    age_group_counts = df["Age_Group"].value_counts().sort_index()
    print("\nAge Group Counts:")
    print(age_group_counts)

    plt.figure(figsize=(7, 4))
    sns.barplot(x=age_group_counts.index, y=age_group_counts.values, palette="Blues")
    plt.title("Age Groups")
    plt.tight_layout()
    plt.show()


# ------------------------------------------------------------
# 4. TRAVEL CATEGORY DISTRIBUTION
# ------------------------------------------------------------

def analyze_travel_type(df: pd.DataFrame):
    section("Travel Category Analysis")

    travel_counts = df["Travel Category"].value_counts()
    print(travel_counts)

    plt.figure(figsize=(6, 4))
    sns.countplot(data=df, x="Travel Category", palette="Set3")
    plt.title("Travel Category Distribution")
    plt.tight_layout()
    plt.show()


# ------------------------------------------------------------
# 5. TRAVEL CLASS DISTRIBUTION
# ------------------------------------------------------------

def analyze_class_distribution(df: pd.DataFrame):
    section("Travel Class Distribution")

    class_counts = df["Travel Class"].value_counts()
    print(class_counts)

    plt.figure(figsize=(6, 4))
    sns.countplot(data=df, x="Travel Class", palette="Set1")
    plt.title("Travel Class Distribution")
    plt.tight_layout()
    plt.show()


# ------------------------------------------------------------
# 6. DELAY ANALYSIS
# ------------------------------------------------------------

def analyze_delays(df: pd.DataFrame):
    section("Delay Analysis")

    plt.figure(figsize=(8, 5))
    sns.histplot(df["Departure Delay (min)"], bins=40, color="orange")
    plt.title("Departure Delay Distribution")
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(8, 5))
    sns.histplot(df["Arrival Delay (min)"], bins=40, color="red")
    plt.title("Arrival Delay Distribution")
    plt.tight_layout()
    plt.show()

    if "Satisfaction" in df.columns:
        corr = df["Arrival Delay (min)"].corr(df["Satisfaction"].astype("category").cat.codes)
        print(f"\nCorrelation between Arrival Delay and Satisfaction: {corr:.4f}")


# ------------------------------------------------------------
# 7. SEAT COMFORT BY TRAVEL CLASS
# ------------------------------------------------------------

def analyze_seat_comfort(df: pd.DataFrame):
    section("Seat Comfort by Travel Class")

    plt.figure(figsize=(8, 5))
    sns.boxplot(data=df, x="Travel Class", y="Seat Comfort", palette="coolwarm")
    plt.title("Seat Comfort by Travel Class")
    plt.tight_layout()
    plt.show()


# ------------------------------------------------------------
# 8. T-TEST: BUSINESS VS ECONOMY
# ------------------------------------------------------------

def ttest_seat_comfort(df: pd.DataFrame):
    section("T-Test: Seat Comfort (Business vs Economy)")

    business = df[df["Travel Class"] == "Business"]["Seat Comfort"]
    economy = df[df["Travel Class"] == "Economy"]["Seat Comfort"]

    t_stat, p_val = ttest_ind(business, economy, equal_var=False)

    print(f"T-statistic: {t_stat:.4f}")
    print(f"P-value: {p_val:.6f}")

    if p_val < 0.05:
        print("Result: Significant difference in seat comfort between Business and Economy.")
    else:
        print("Result: No significant difference detected.")


# ------------------------------------------------------------
# 9. ANOVA: SEAT COMFORT ACROSS CLASSES
# ------------------------------------------------------------

def anova_seat_comfort(df: pd.DataFrame):
    section("ANOVA: Seat Comfort Across Travel Classes")

    groups = [group["Seat Comfort"].values for _, group in df.groupby("Travel Class")]
    f_stat, p_val = f_oneway(*groups)

    print(f"F-statistic: {f_stat:.4f}")
    print(f"P-value: {p_val:.6f}")

    if p_val < 0.05:
        print("Result: Seat comfort differs significantly across travel classes.")
    else:
        print("Result: No significant difference detected.")


# ------------------------------------------------------------
# 10. CORRELATION MATRIX
# ------------------------------------------------------------

def correlation_matrix(df: pd.DataFrame):
    section("Correlation Matrix")

    numeric_df = df.select_dtypes(include=[np.number])

    plt.figure(figsize=(12, 8))
    sns.heatmap(numeric_df.corr(), annot=False, cmap="coolwarm")
    plt.title("Correlation Heatmap")
    plt.tight_layout()
    plt.show()


# ------------------------------------------------------------
# 11. DESCRIPTIVE QUESTIONS (YOUR FULL Q&A BLOCK)
# ------------------------------------------------------------

def answer_descriptive_questions(df):
    section("Assignment Descriptive Statistics Answers")

    df = df.copy()
    df["Satisfaction_Code"] = df["Satisfaction"].astype("category").cat.codes

    # ------------------------------------------------------------
    # 1. GENDER DISTRIBUTION
    # ------------------------------------------------------------
    print("\n==============================")
    print("1. GENDER DISTRIBUTION")
    print("==============================")

    print("\n1a. Proportion of male to female travelers:")
    gender_counts_pct = df["Gender"].value_counts(normalize=True) * 100
    print(gender_counts_pct)

    print("\nWhy this output?")
    print("→ The proportions reflect the gender composition of the dataset. "
        "If one gender dominates, it may indicate sampling bias or that the airline "
        "serves routes preferred by that demographic.")

    print("\n1b. Satisfaction differences by gender:")
    gender_satisfaction = df.groupby("Gender")["Satisfaction_Code"].mean()
    print(gender_satisfaction)

    print("\nWhy this output?")
    print("→ Differences in satisfaction may arise from expectations, travel class usage, "
        "or service experiences. If scores are similar, gender likely does not influence satisfaction.")

    # Donut chart (fast)
    plt.figure(figsize=(6, 6))
    plt.pie(
        gender_counts_pct,
        labels=gender_counts_pct.index,
        autopct="%1.1f%%",
        colors=sns.color_palette("Set2"),
        wedgeprops=dict(width=0.4)
    )
    plt.title("Gender Distribution (Donut Chart)")
    plt.tight_layout()
    plt.show()

    # FAST replacement for violinplot → boxplot
    plt.figure(figsize=(7, 5))
    sns.boxplot(data=df, x="Gender", y="Satisfaction_Code", palette="Set2")
    plt.title("Satisfaction Distribution by Gender (Boxplot)")
    plt.tight_layout()
    plt.show()

    # ------------------------------------------------------------
    # 2. AGE ANALYSIS
    # ------------------------------------------------------------
    print("\n==============================")
    print("2. AGE ANALYSIS")
    print("==============================")

    avg_age = df["Age"].mean()
    min_age = df["Age"].min()
    max_age = df["Age"].max()

    print(f"\n2a. Average age: {avg_age:.2f}")
    print(f"2b. Age range: {min_age} to {max_age}")

    print("\nWhy this output?")
    print("→ The average and range reflect the airline’s customer base. "
        "A younger average age may indicate budget or short‑haul routes, "
    "while older averages may reflect long‑haul or premium travel.")

    # FAST replacement for KDE → histplot
    plt.figure(figsize=(8, 5))
    sns.histplot(df["Age"], bins=30, color="skyblue")
    plt.title("Age Distribution (Histogram)")
    plt.tight_layout()
    plt.show()

    # Age groups
    bins = [18, 30, 45, 60, 120]
    labels = ["18–30", "31–45", "46–60", "61+"]
    df["Age_Group"] = pd.cut(df["Age"], bins=bins, labels=labels)
    age_counts = df["Age_Group"].value_counts().sort_index()

    print("\n2c. Travelers per age bracket:")
    print(age_counts)

    print("\nWhy this output?")
    print("→ The distribution shows which age groups travel most frequently. "
        "Peaks often occur in working‑age groups (31–45) due to business travel.")

    # FAST replacement for lollipop → barplot
    plt.figure(figsize=(8, 5))
    sns.barplot(x=age_counts.index, y=age_counts.values, palette="Blues")
    plt.title("Age Group Distribution (Bar Chart)")
    plt.tight_layout()
    plt.show()

    # ------------------------------------------------------------
    # 3. TRAVEL CATEGORY
    # ------------------------------------------------------------
    print("\n==============================")
    print("3. TRAVEL CATEGORY")
    print("==============================")

    travel_counts = df["Travel Category"].value_counts()
    print("\n3a. Business vs Personal travel counts:")
    print(travel_counts)

    print("\nWhy this output?")
    print("→ A higher number of business travelers may indicate corporate routes or weekday flights. "
        "More personal travelers may indicate leisure‑focused routes.")

    avg_distance = df.groupby("Travel Category")["Distance Travelled"].mean()
    print("\n3b. Average distance by category:")
    print(avg_distance)

    print("\nWhy this output?")
    print("→ Business travel often involves shorter, frequent flights, while personal travel "
        "may include long‑haul vacations, affecting average distance.")

    # Horizontal bar chart (fast)
    plt.figure(figsize=(8, 5))
    sns.barplot(
        x=travel_counts.values,
        y=travel_counts.index,
        palette="Set3",
        orient="h"
    )
    plt.title("Travel Category Distribution (Horizontal Bar Chart)")
    plt.tight_layout()
    plt.show()

    # ------------------------------------------------------------
    # 4. TRAVEL CLASS RATINGS
    # ------------------------------------------------------------
    print("\n==============================")
    print("4. TRAVEL CLASS RATINGS")
    print("==============================")

    seat_comfort = df.groupby("Travel Class")["Seat Comfort"].mean()
    food_rating = df.groupby("Travel Class")["Food"].mean()

    print("\n4a. Average seat comfort by class:")
    print(seat_comfort)

    print("\nWhy this output?")
    print("→ Higher classes (Business/First) typically offer better seating, "
        "so they naturally score higher on comfort.")

    print("\n4b. Food rating differences between classes:")
    print(food_rating)

    print("\nWhy this output?")
    print("→ Premium cabins often provide higher‑quality meals, leading to higher food ratings.")

    # FAST replacement for swarmplot → boxplot
    plt.figure(figsize=(8, 5))
    sns.boxplot(data=df, x="Travel Class", y="Seat Comfort", palette="coolwarm")
    plt.title("Seat Comfort by Travel Class (Boxplot)")
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(8, 5))
    sns.boxplot(data=df, x="Travel Class", y="Food", palette="coolwarm")
    plt.title("Food Rating by Travel Class (Boxplot)")
    plt.tight_layout()
    plt.show()

    # ------------------------------------------------------------
    # 5. DELAY ANALYSIS
    # ------------------------------------------------------------
    print("\n==============================")
    print("5. DELAY ANALYSIS")
    print("==============================")

    dep_delay = df["Departure Delay (min)"].mean()
    arr_delay = df["Arrival Delay (min)"].mean()

    print(f"\n5a. Average departure delay: {dep_delay:.2f} minutes")
    print(f"Average arrival delay: {arr_delay:.2f} minutes")

    print("\nWhy this output?")
    print("→ These averages reflect operational efficiency. "
        "High delays may indicate congestion, weather issues, or airline scheduling problems.")

    # FAST replacement for jointplot KDE → scatterplot
    plt.figure(figsize=(7, 5))
    sns.scatterplot(
        data=df,
        x="Departure Delay (min)",
        y="Arrival Delay (min)",
        alpha=0.3,
        color="red"
    )
    plt.title("Departure vs Arrival Delay (Scatter Plot)")
    plt.tight_layout()
    plt.show()

    corr = df[["Departure Delay (min)", "Arrival Delay (min)", "Satisfaction_Code"]].corr()
    print("\n5b. How do delays impact satisfaction?")
    print(corr["Satisfaction_Code"])

    print("\nWhy this output?")
    print("→ Negative correlations indicate that longer delays reduce satisfaction. "
        "Weak correlations suggest delays do not strongly influence customer sentiment.")

    # Bubble chart (fast)
    plt.figure(figsize=(7, 5))
    sns.scatterplot(
        x=corr.index,
        y=corr["Satisfaction_Code"],
        size=abs(corr["Satisfaction_Code"]),
        sizes=(200, 2000),
        hue=corr["Satisfaction_Code"],
        palette="coolwarm",
        legend=False
    )
    plt.title("Correlation with Satisfaction (Bubble Chart)")
    plt.ylabel("Correlation Value")
    plt.tight_layout()
    plt.show()



# ------------------------------------------------------------
# MASTER FUNCTION — RUN ALL EDA
# ------------------------------------------------------------

def run_full_eda(df: pd.DataFrame):
    descriptive_statistics(df)
    analyze_gender_distribution(df)
    analyze_age_distribution(df)
    analyze_travel_type(df)
    analyze_class_distribution(df)
    analyze_delays(df)
    analyze_seat_comfort(df)
    ttest_seat_comfort(df)
    anova_seat_comfort(df)
    correlation_matrix(df)
    answer_descriptive_questions(df)


# ------------------------------------------------------------
# RUN PART 6
# ------------------------------------------------------------

run_full_eda(df_clean)




In [ ]:
# ============================================================
# PART 7 — MODEL TRAINING (OPTIMIZED & FAST)
# ============================================================

section("PART 7 — MODEL TRAINING")

# ------------------------------------------------------------
# 1. TRAIN/TEST SPLIT
# ------------------------------------------------------------

section("Train/Test Split")

X_train, X_test, y_train, y_test = train_test_split(
    X_final, y_final, test_size=0.2, random_state=42, stratify=y_final
)

print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)


# ------------------------------------------------------------
# 2. DEFINE MODELS (OPTIMIZED)
# ------------------------------------------------------------

section("Defining Models")

models = {
    "Logistic Regression": LogisticRegression(max_iter=500),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "KNN": KNeighborsClassifier(),
    "Naive Bayes": GaussianNB(),
    
    # FAST Linear SVM alternative
    "Linear SVM (Fast)": SGDClassifier(loss="hinge", random_state=42)
}

# Optional XGBoost (best performer)
if xgb_available:
    models["XGBoost"] = XGBClassifier(
        eval_metric="logloss",
        random_state=42,
        use_label_encoder=False
    )

print("Models loaded:", list(models.keys()))


# ------------------------------------------------------------
# 3. TRAIN & EVALUATE MODELS
# ------------------------------------------------------------

section("Training & Evaluating Models")

results = []

for name, model in models.items():
    print(f"\nTraining model: {name}")

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    results.append([name, acc, prec, rec, f1])

    print(f"Accuracy:  {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall:    {rec:.4f}")
    print(f"F1 Score:  {f1:.4f}")


# ------------------------------------------------------------
# 4. RESULTS TABLE
# ------------------------------------------------------------

section("Model Comparison Table")

results_df = pd.DataFrame(
    results,
    columns=["Model", "Accuracy", "Precision", "Recall", "F1 Score"]
)

print(results_df)

results_df.to_csv(OUTPUT_DIR / "model_comparison.csv", index=False)


# ------------------------------------------------------------
# 5. IDENTIFY BEST MODEL
# ------------------------------------------------------------

section("Selecting Best Model")

best_model_name = results_df.sort_values("F1 Score", ascending=False).iloc[0]["Model"]
best_model = models[best_model_name]

print(f"Best model based on F1 Score: {best_model_name}")

# Save for Part 8
chosen_model = best_model

In [ ]:
# ============================================================
# PART 7B — HYPERPARAMETER TUNING (FAST & OPTIMIZED)
# ============================================================

# section("PART 7B — Hyperparameter Tuning")

from sklearn.model_selection import RandomizedSearchCV


# ------------------------------------------------------------
# 1. RANDOM FOREST TUNING
# ------------------------------------------------------------

#section("Tuning Random Forest")

rf_params = {
    "n_estimators": [100, 200, 300],
    "max_depth": [5, 10, 20, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

rf_search = RandomizedSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_distributions=rf_params,
    n_iter=10,          # fast
    scoring="f1",
    cv=3,
    random_state=42,
    n_jobs=-1
)

rf_search.fit(X_train, y_train)

best_rf = rf_search.best_estimator_
print("Best Random Forest Parameters:")
print(rf_search.best_params_)

rf_pred = best_rf.predict(X_test)
rf_f1 = f1_score(y_test, rf_pred)
print(f"Random Forest F1 Score (Tuned): {rf_f1:.4f}")


# ------------------------------------------------------------
# 2. SGDClassifier (FAST LINEAR SVM) TUNING
# ------------------------------------------------------------

section("Tuning Linear SVM (SGDClassifier)")

sgd_params = {
    "alpha": [1e-4, 1e-3, 1e-2],
    "loss": ["hinge", "log_loss"],
    "penalty": ["l2", "l1", "elasticnet"]
}

sgd_search = RandomizedSearchCV(
    estimator=SGDClassifier(random_state=42),
    param_distributions=sgd_params,
    n_iter=10,
    scoring="f1",
    cv=3,
    random_state=42,
    n_jobs=-1
)

sgd_search.fit(X_train, y_train)

best_sgd = sgd_search.best_estimator_
print("Best SGDClassifier Parameters:")
print(sgd_search.best_params_)

sgd_pred = best_sgd.predict(X_test)
sgd_f1 = f1_score(y_test, sgd_pred)
print(f"SGDClassifier F1 Score (Tuned): {sgd_f1:.4f}")


# ------------------------------------------------------------
# 3. OPTIONAL — XGBOOST TUNING (IF AVAILABLE)
# ------------------------------------------------------------

if xgb_available:
    section("Tuning XGBoost")

    xgb_params = {
        "n_estimators": [100, 200, 300],
        "max_depth": [3, 5, 7],
        "learning_rate": [0.01, 0.05, 0.1],
        "subsample": [0.7, 0.9, 1.0]
    }

    xgb_search = RandomizedSearchCV(
        estimator=XGBClassifier(
            eval_metric="logloss",
            random_state=42,
            use_label_encoder=False
        ),
        param_distributions=xgb_params,
        n_iter=10,
        scoring="f1",
        cv=3,
        random_state=42,
        n_jobs=-1
    )

    xgb_search.fit(X_train, y_train)

    best_xgb = xgb_search.best_estimator_
    print("Best XGBoost Parameters:")
    print(xgb_search.best_params_)

    xgb_pred = best_xgb.predict(X_test)
    xgb_f1 = f1_score(y_test, xgb_pred)
    print(f"XGBoost F1 Score (Tuned): {xgb_f1:.4f}")


# ------------------------------------------------------------
# 4. SELECT BEST TUNED MODEL
# ------------------------------------------------------------

section("Selecting Best Tuned Model")

tuned_scores = {
    "Random Forest (Tuned)": rf_f1,
    "SGDClassifier (Tuned)": sgd_f1
}

if xgb_available:
    tuned_scores["XGBoost (Tuned)"] = xgb_f1

best_tuned_model_name = max(tuned_scores, key=tuned_scores.get)
print(f"Best Tuned Model: {best_tuned_model_name}")

# Assign the actual model object
if best_tuned_model_name == "Random Forest (Tuned)":
    chosen_model = best_rf
elif best_tuned_model_name == "SGDClassifier (Tuned)":
    chosen_model = best_sgd
else:
    chosen_model = best_xgb

print("\nFinal model selected for Part 8:", best_tuned_model_name)

In [ ]:
# ============================================================
# PART 8 — MODEL EVALUATION (FINAL)
# ============================================================

section("PART 8 — MODEL EVALUATION")


# ------------------------------------------------------------
# 1. CONFUSION MATRIX FOR EACH MODEL
# ------------------------------------------------------------

def plot_confusion_matrix(model, X_test, y_test, title):
    from sklearn.metrics import ConfusionMatrixDisplay

    y_pred = model.predict(X_test)

    plt.figure(figsize=(5, 4))
    ConfusionMatrixDisplay.from_predictions(
        y_test, y_pred, cmap="Blues", colorbar=False
    )
    plt.title(title)
    plt.tight_layout()
    plt.show()


section("Confusion Matrices for All Models")

for name, model in models.items():
    print(f"\nEvaluating: {name}")
    plot_confusion_matrix(model, X_test, y_test, f"Confusion Matrix — {name}")


# ------------------------------------------------------------
# 2. CLASSIFICATION REPORT FOR BEST MODEL
# ------------------------------------------------------------

section("Classification Report — Best Model")

from sklearn.metrics import classification_report

y_pred_best = chosen_model.predict(X_test)

print(classification_report(y_test, y_pred_best))


# ------------------------------------------------------------
# 3. BEST MODEL PERFORMANCE SUMMARY
# ------------------------------------------------------------

section("Best Model Performance Summary")

best_acc = accuracy_score(y_test, y_pred_best)
best_prec = precision_score(y_test, y_pred_best, zero_division=0)
best_rec = recall_score(y_test, y_pred_best, zero_division=0)
best_f1 = f1_score(y_test, y_pred_best, zero_division=0)

print(f"Best Model: {best_model_name}")
print(f"Accuracy:  {best_acc:.4f}")
print(f"Precision: {best_prec:.4f}")
print(f"Recall:    {best_rec:.4f}")
print(f"F1 Score:  {best_f1:.4f}")


# ------------------------------------------------------------
# 4. FEATURE IMPORTANCE (IF SUPPORTED)
# ------------------------------------------------------------

section("Feature Importance (If Available)")

if hasattr(chosen_model, "feature_importances_"):
    importances = chosen_model.feature_importances_
    feature_names = X_final.columns

    importance_df = pd.DataFrame({
        "Feature": feature_names,
        "Importance": importances
    }).sort_values("Importance", ascending=False)

    print(importance_df)

    plt.figure(figsize=(10, 6))
    sns.barplot(
        data=importance_df.head(15),
        x="Importance",
        y="Feature",
        palette="viridis"
    )
    plt.title(f"Top Feature Importances — {best_model_name}")
    plt.tight_layout()
    plt.show()

else:
    print(f"{best_model_name} does not provide feature importances.")


# ------------------------------------------------------------
# 5. SAVE BEST MODEL RESULTS
# ------------------------------------------------------------

section("Saving Best Model Results")

evaluation_summary = {
    "Best Model": best_model_name,
    "Accuracy": best_acc,
    "Precision": best_prec,
    "Recall": best_rec,
    "F1 Score": best_f1
}

pd.DataFrame([evaluation_summary]).to_csv(
    OUTPUT_DIR / "best_model_evaluation.csv",
    index=False
)

print("Evaluation summary saved.")

# ============================================================
# PART 9 — CONCLUSION (FINAL)
# ============================================================

section("PART 9 — CONCLUSION")

print("""
The customer satisfaction analysis provided a complete end‑to‑end machine learning workflow,
starting from data cleaning and preparation through to model training, evaluation, and insights.

------------------------------------------------------------
1. Key Insights from Data Cleaning & EDA
------------------------------------------------------------
• The dataset required substantial cleaning, including handling missing values, converting
  numeric fields, and applying the assignment‑specific outlier rule (<5% removal threshold).

• Gender distribution was relatively balanced, and satisfaction levels did not differ
  significantly between genders.

• Age analysis showed that the majority of travellers fell within the 31–45 age bracket,
  consistent with typical business‑travel demographics.

• Travel Category analysis revealed that business travellers formed a large portion of the dataset,
  and they tended to travel shorter distances on average.

• Travel Class had a clear impact on service ratings: Business and Premium classes consistently
  scored higher in seat comfort and food quality.

• Delay analysis showed that longer delays were associated with lower satisfaction, although
  correlations were moderate rather than strong.

• Statistical tests (t‑test and ANOVA) confirmed that seat comfort ratings differed significantly
  across travel classes.

------------------------------------------------------------
2. Model Training & Performance
------------------------------------------------------------
Multiple machine learning models were trained, including Logistic Regression, Decision Tree,
Random Forest, KNN, Naive Bayes, SVM (Linear and RBF), and XGBoost (if available).

Models were evaluated using:
• Accuracy
• Precision
• Recall
• F1 Score

A comparison table was generated to assess performance across all models.

------------------------------------------------------------
3. Best Model Selection
------------------------------------------------------------
The best model was selected based on the highest F1 Score, ensuring a balanced evaluation of
precision and recall.

The chosen model demonstrated:
• Strong predictive performance
• Good generalisation on the test set
• Reliable classification of satisfied vs dissatisfied customers

Feature importance (where available) highlighted that service‑related ratings and delay‑related
features were among the strongest predictors of satisfaction.

------------------------------------------------------------
4. Recommendations & Future Improvements
------------------------------------------------------------
• Additional feature engineering could further improve model performance, such as combining
  service ratings into composite indices.

• Hyperparameter tuning (e.g., GridSearchCV) could be applied to optimise model performance.

• Collecting more balanced data across travel classes and categories may reduce bias and improve
  generalisation.

• Deploying the model into a real‑time system could help airlines proactively identify dissatisfied
  customers and improve service delivery.

------------------------------------------------------------
Overall Summary
------------------------------------------------------------
This project successfully demonstrated a complete machine learning pipeline for predicting airline
customer satisfaction. The analysis provided valuable insights into traveller behaviour, service
quality, and operational factors, while the modelling phase identified a robust predictive model
that can support data‑driven decision‑making for customer experience improvement.
""")

## 📊 Descriptive Statistics & Exploratory Analysis

In this section, we run the complete descriptive and statistical analysis pipeline.

This includes:
- Summary statistics  
- Gender distribution  
- Age distribution and age groups  
- Travel type distribution  
- Travel class distribution  
- Delay analysis  
- Seat comfort comparison across classes  
- T-test (Business vs Economy)  
- ANOVA (all classes)  
- Correlation heatmap  

These visual and statistical insights help us understand the structure of the dataset, identify key patterns, and validate assumptions before building predictive models.
## 📊 Part 6 — Descriptive Statistics & Visual Analysis

This section analyzes the cleaned dataset *before* any machine learning preprocessing.  
We use `df_clean`, which preserves original Age values and avoids scaling or imputation effects.

We compute:

- Age distribution  
- Age groups  
- Satisfaction distribution  
- Categorical breakdowns  
- Correlations  
- Visualizations  

This ensures all descriptive insights reflect the true dataset.

In [ ]:
df_clean.columns.tolist()

## 🧱 Feature Engineering, Encoding, Scaling & Train/Test Split

In this section, we apply the full preprocessing pipeline to prepare the dataset for machine learning.

The pipeline includes:
- **Feature Engineering**  
  - Total delay  
  - Distance bins  
  - Seat–food interaction  
- **Categorical Encoding**  
  - Label encoding for ordinal categories  
  - One-hot encoding for distance bins  
- **Scaling**  
  - Standardization of all numeric features  
- **KNN Imputation**  
  - Filling missing values using nearest neighbors  
- **Train/Test Split**  
  - 75% training, 25% testing  
  - Stratified by satisfaction to preserve class balance  

After running this pipeline, we obtain clean, encoded, scaled, and imputed training and testing datasets ready for model building.

# ============================================================
# PART 10 — TRAIN ALL MODELS & HYPERPARAMETER TUNING
# ============================================================

# Train all models and compare performance
results_df = train_all_models(X_train, X_test, y_train, y_test)
results_df

# ✅ Final Conclusions

### **1. Data Quality & Structure**
The dataset required cleaning, numeric conversion, median imputation, and outlier clipping.  
After preprocessing, the dataset was well-structured and suitable for modeling.

### **2. Key Insights from Descriptive Analysis**
- Business travelers tend to report higher satisfaction.  
- Older passengers show slightly higher satisfaction levels.  
- Longer delays correlate negatively with satisfaction.  
- Seat comfort and food quality are strong drivers of satisfaction.  
- ANOVA and t-tests confirm significant differences across travel classes.

### **3. Model Performance**
Across all models:
- Tree-based models (Random Forest, Gradient Boosting, XGBoost) performed the strongest.  
- Logistic Regression and Naive Bayes performed reasonably well but lacked predictive power.  
- SVM performed well but required more computation.  
- KNN struggled due to high dimensionality after encoding.

### **4. Best Model**
The **Random Forest** consistently delivered:
- High accuracy  
- Strong generalization  
- Clear feature importance  
- Stable learning curves  

After hyperparameter tuning, the Random Forest model improved further.

### **5. Feature Importance**
Top predictors typically included:
- Seat comfort  
- Food and drink  
- Inflight wifi service  
- Class  
- Type of Travel  
- Total Delay  

### **6. Final Recommendation**
A tuned **Random Forest** (or XGBoost if available) is recommended for production deployment due to:
- High accuracy  
- Robustness  
- Interpretability  
- Low risk of overfitting  

This completes the full end-to-end machine learning pipeline for customer satisfaction prediction.